In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1308269,2021-05-25,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1308270,2021-05-26,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1308271,2021-05-27,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1308272,2021-05-28,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
34767,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
34769,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
34771,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
34773,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
34775,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
803370,2021-05-25,Arkansas,Arkansas,5001,2118,37.00,5000,Arkansas,AR,Arkansas,State,3017804
803372,2021-05-26,Arkansas,Arkansas,5001,2118,37.00,5000,Arkansas,AR,Arkansas,State,3017804
803374,2021-05-27,Arkansas,Arkansas,5001,2118,37.00,5000,Arkansas,AR,Arkansas,State,3017804
803376,2021-05-28,Arkansas,Arkansas,5001,2123,37.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1308269,2021-05-25,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1308270,2021-05-26,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1308271,2021-05-27,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1308272,2021-05-28,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-05-29') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
494,2021-05-29,Snohomish,Washington,38872,585.00,53061,WA,County,822083,4728.48,71.16
986,2021-05-29,Cook,Illinois,553191,10840.00,17031,IL,County,5150233,10741.09,210.48
1477,2021-05-29,Orange,California,272059,5063.00,6059,CA,County,3175692,8566.92,159.43
1967,2021-05-29,Maricopa,Arizona,550067,10069.00,4013,AZ,County,4485414,12263.46,224.48
2457,2021-05-29,Los Angeles,California,1244380,24357.00,6037,CA,County,10039107,12395.33,242.62
...,...,...,...,...,...,...,...,...,...,...,...
1307481,2021-05-29,Wheeler,Oregon,34,1.00,41069,OR,County,1332,2552.55,75.08
1307710,2021-05-29,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1307908,2021-05-29,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1308102,2021-05-29,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
494,2021-05-29,Snohomish,Washington,38872,585.00,53061,WA,County,822083,4728.48,71.16,71.16,4728.48
986,2021-05-29,Cook,Illinois,553191,10840.00,17031,IL,County,5150233,10741.09,210.48,210.48,10741.09
1477,2021-05-29,Orange,California,272059,5063.00,6059,CA,County,3175692,8566.92,159.43,159.43,8566.92
1967,2021-05-29,Maricopa,Arizona,550067,10069.00,4013,AZ,County,4485414,12263.46,224.48,224.48,12263.46
2457,2021-05-29,Los Angeles,California,1244380,24357.00,6037,CA,County,10039107,12395.33,242.62,242.62,12395.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1307481,2021-05-29,Wheeler,Oregon,34,1.00,41069,OR,County,1332,2552.55,75.08,75.08,2552.55
1307710,2021-05-29,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1307908,2021-05-29,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1308102,2021-05-29,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00,0.00,1183.43


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
840097,2021-05-29,Hale,Alabama,2252,78.00,1065,AL,County,14651,15370.96,532.39,532.39,15370.96,1
914567,2021-05-29,Clarke,Alabama,3521,61.00,1025,AL,County,23622,14905.60,258.23,258.23,14905.60,2
657081,2021-05-29,Lowndes,Alabama,1412,54.00,1085,AL,County,9726,14517.79,555.21,555.21,14517.79,3
582524,2021-05-29,Etowah,Alabama,13981,359.00,1055,AL,County,102268,13670.94,351.04,351.04,13670.94,4
488706,2021-05-29,Franklin,Alabama,4273,82.00,1059,AL,County,31362,13624.77,261.46,261.46,13624.77,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1238865,2021-05-29,Platte,Wyoming,651,11.00,56031,WY,County,8393,7756.46,131.06,131.06,7756.46,19
801669,2021-05-29,Converse,Wyoming,1013,17.00,56009,WY,County,13822,7328.90,122.99,122.99,7328.90,20
997054,2021-05-29,Lincoln,Wyoming,1413,12.00,56023,WY,County,19830,7125.57,60.51,60.51,7125.57,21
1110000,2021-05-29,Niobrara,Wyoming,162,2.00,56027,WY,County,2356,6876.06,84.89,84.89,6876.06,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
657081,2021-05-29,Lowndes,Alabama,1412,54.00,1085,AL,County,9726,14517.79,555.21,555.21,14517.79,3,1
840097,2021-05-29,Hale,Alabama,2252,78.00,1065,AL,County,14651,15370.96,532.39,532.39,15370.96,1,2
279191,2021-05-29,Walker,Alabama,7284,278.00,1127,AL,County,63521,11467.07,437.65,437.65,11467.07,28,3
580800,2021-05-29,Clay,Alabama,1578,57.00,1027,AL,County,13235,11922.93,430.68,430.68,11922.93,20,4
708678,2021-05-29,Greene,Alabama,928,34.00,1063,AL,County,8111,11441.25,419.18,419.18,11441.25,29,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706961,2021-05-29,Hot Springs,Wyoming,370,3.00,56017,WY,County,4413,8384.32,67.98,67.98,8384.32,17,19
972156,2021-05-29,Uinta,Wyoming,2266,13.00,56041,WY,County,20226,11203.40,64.27,64.27,11203.40,5,20
997054,2021-05-29,Lincoln,Wyoming,1413,12.00,56023,WY,County,19830,7125.57,60.51,60.51,7125.57,21,21
654501,2021-05-29,Albany,Wyoming,4516,15.00,56001,WY,County,38880,11615.23,38.58,38.58,11615.23,4,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,71.16,4728.48,20,25
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,71.16,4728.48,20,25
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,71.16,4728.48,20,25
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,71.16,4728.48,20,25
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,71.16,4728.48,20,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305210,2021-05-25,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1305211,2021-05-26,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1305212,2021-05-27,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1305213,2021-05-28,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
836613,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15370.96,2,1,1.00
836614,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15370.96,2,1,0.00
836615,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15370.96,2,1,0.00
836616,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15370.96,2,1,0.00
836617,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,532.39,15370.96,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1013778,2021-05-25,Crook,Wyoming,440,12.00,56011,WY,County,7584,5801.69,158.23,158.23,5880.80,6,23,2.00
1013779,2021-05-26,Crook,Wyoming,441,12.00,56011,WY,County,7584,5814.87,158.23,158.23,5880.80,6,23,1.00
1013780,2021-05-27,Crook,Wyoming,442,12.00,56011,WY,County,7584,5828.06,158.23,158.23,5880.80,6,23,1.00
1013781,2021-05-28,Crook,Wyoming,446,12.00,56011,WY,County,7584,5880.80,158.23,158.23,5880.80,6,23,4.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
654449,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14517.79,1,3,1.00,0.00
654450,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14517.79,1,3,0.00,0.00
654451,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14517.79,1,3,0.00,0.00
654452,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14517.79,1,3,0.00,0.00
654453,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14517.79,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276104,2021-05-25,Teton,Wyoming,3780,9.00,56039,WY,County,23464,16109.79,38.36,38.36,16126.83,23,1,0.00,0.00
276105,2021-05-26,Teton,Wyoming,3781,9.00,56039,WY,County,23464,16114.05,38.36,38.36,16126.83,23,1,1.00,0.00
276106,2021-05-27,Teton,Wyoming,3781,9.00,56039,WY,County,23464,16114.05,38.36,38.36,16126.83,23,1,0.00,0.00
276107,2021-05-28,Teton,Wyoming,3784,9.00,56039,WY,County,23464,16126.83,38.36,38.36,16126.83,23,1,3.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-05-29') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
333347,2021-05-29,Imperial,California,28722,735.00,6025,CA,County,181215,15849.68,405.60,405.60,15849.68,1,2,-6.00,0.00
2457,2021-05-29,Los Angeles,California,1244380,24357.00,6037,CA,County,10039107,12395.33,242.62,242.62,12395.33,2,5,192.00,9.00
165871,2021-05-29,San Bernardino,California,298289,4757.00,6071,CA,County,2180085,13682.45,218.20,218.20,13682.45,3,4,0.00,1.00
584200,2021-05-29,Inyo,California,1430,38.00,6027,CA,County,18039,7927.27,210.65,210.65,7927.27,4,25,0.00,0.00
74117,2021-05-29,Stanislaus,California,62788,1066.00,6099,CA,County,550660,11402.32,193.59,193.59,11402.32,5,9,25.00,0.00
34317,2021-05-29,Riverside,California,300701,4608.00,6065,CA,County,2470546,12171.44,186.52,186.52,12171.44,6,7,0.00,0.00
58955,2021-05-29,San Joaquin,California,73794,1419.00,6077,CA,County,762148,9682.37,186.18,186.18,9682.37,7,17,0.00,0.00
91914,2021-05-29,Tulare,California,49934,846.00,6107,CA,County,466195,10710.97,181.47,181.47,10710.97,8,10,0.00,0.00
33419,2021-05-29,Fresno,California,102437,1705.00,6019,CA,County,999101,10252.92,170.65,170.65,10252.92,9,14,24.00,1.00
529353,2021-05-29,Merced,California,32142,469.00,6047,CA,County,277680,11575.19,168.90,168.90,11575.19,10,8,0.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1238416,2021-05-29,Lassen,California,5736,24.00,6035,CA,County,30573,18761.65,78.50,78.50,18761.65,35,1,0.00,0.00
333347,2021-05-29,Imperial,California,28722,735.00,6025,CA,County,181215,15849.68,405.60,405.60,15849.68,1,2,-6.00,0.00
708620,2021-05-29,Kings,California,23067,247.00,6031,CA,County,152940,15082.39,161.50,161.50,15082.39,11,3,8.00,0.00
165871,2021-05-29,San Bernardino,California,298289,4757.00,6071,CA,County,2180085,13682.45,218.20,218.20,13682.45,3,4,0.00,1.00
2457,2021-05-29,Los Angeles,California,1244380,24357.00,6037,CA,County,10039107,12395.33,242.62,242.62,12395.33,2,5,192.00,9.00
208593,2021-05-29,Kern,California,110089,1389.00,6029,CA,County,900202,12229.37,154.30,154.30,12229.37,14,6,90.00,1.00
34317,2021-05-29,Riverside,California,300701,4608.00,6065,CA,County,2470546,12171.44,186.52,186.52,12171.44,6,7,0.00,0.00
529353,2021-05-29,Merced,California,32142,469.00,6047,CA,County,277680,11575.19,168.90,168.90,11575.19,10,8,0.00,0.00
74117,2021-05-29,Stanislaus,California,62788,1066.00,6099,CA,County,550660,11402.32,193.59,193.59,11402.32,5,9,25.00,0.00
91914,2021-05-29,Tulare,California,49934,846.00,6107,CA,County,466195,10710.97,181.47,181.47,10710.97,8,10,0.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
333347,2021-05-29,Imperial,California,28722,735.00,6025,CA,County,181215,15849.68,405.60,405.60,15849.68,1,2,-6.00,0.00
2457,2021-05-29,Los Angeles,California,1244380,24357.00,6037,CA,County,10039107,12395.33,242.62,242.62,12395.33,2,5,192.00,9.00
165871,2021-05-29,San Bernardino,California,298289,4757.00,6071,CA,County,2180085,13682.45,218.20,218.20,13682.45,3,4,0.00,1.00
584200,2021-05-29,Inyo,California,1430,38.00,6027,CA,County,18039,7927.27,210.65,210.65,7927.27,4,25,0.00,0.00
74117,2021-05-29,Stanislaus,California,62788,1066.00,6099,CA,County,550660,11402.32,193.59,193.59,11402.32,5,9,25.00,0.00
34317,2021-05-29,Riverside,California,300701,4608.00,6065,CA,County,2470546,12171.44,186.52,186.52,12171.44,6,7,0.00,0.00
58955,2021-05-29,San Joaquin,California,73794,1419.00,6077,CA,County,762148,9682.37,186.18,186.18,9682.37,7,17,0.00,0.00
91914,2021-05-29,Tulare,California,49934,846.00,6107,CA,County,466195,10710.97,181.47,181.47,10710.97,8,10,0.00,0.00
33419,2021-05-29,Fresno,California,102437,1705.00,6019,CA,County,999101,10252.92,170.65,170.65,10252.92,9,14,24.00,1.00
529353,2021-05-29,Merced,California,32142,469.00,6047,CA,County,277680,11575.19,168.90,168.90,11575.19,10,8,0.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,405.60,15849.68,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,405.60,15849.68,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,405.60,15849.68,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,405.60,15849.68,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,405.60,15849.68,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4372,05/25/21,Lassen,5721,24.00,18712.59,78.50,78.50,18761.65,35,1,0.00,0.00
4373,05/26/21,Lassen,5726,24.00,18728.94,78.50,78.50,18761.65,35,1,5.00,0.00
4374,05/27/21,Lassen,5736,24.00,18761.65,78.50,78.50,18761.65,35,1,10.00,0.00
4375,05/28/21,Lassen,5736,24.00,18761.65,78.50,78.50,18761.65,35,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,405.60,15849.68,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,405.60,15849.68,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,405.60,15849.68,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,405.60,15849.68,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,405.60,15849.68,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4458,05/25/21,Merced,32093,468.00,11557.55,168.54,168.90,11575.19,10,8,11.00,0.00
4459,05/26/21,Merced,32110,468.00,11563.67,168.54,168.90,11575.19,10,8,17.00,0.00
4460,05/27/21,Merced,32125,468.00,11569.07,168.54,168.90,11575.19,10,8,15.00,0.00
4461,05/28/21,Merced,32142,469.00,11575.19,168.90,168.90,11575.19,10,8,17.00,1.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)